In [67]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
# Sample dataframes structure
users_df = pd.DataFrame({
    'user_id': range(1, 100001),
    'signup_date': pd.date_range(start='2022-01-01', periods=100000, freq='H'),
    'state': np.random.choice(['CA', 'TX', 'NY', 'FL', 'IL'], 100000),
    'age_bucket': np.random.choice(['18-25', '26-35', '36-45', '46-55', '55+'], 100000),
    'is_test_group': np.random.choice([0, 1], 100000, p=[0.5, 0.5]),
    'churn_date': [None] * 100000
})

direct_deposits_df = pd.DataFrame({
    'deposit_id': range(1, 500001),
    'user_id': np.random.choice(range(1, 100001), 500000),
    'deposit_date': pd.date_range(start='2022-03-01', periods=500000, freq='10min'),
    'deposit_amount': np.random.uniform(500, 3000, 500000),
    'days_early': np.random.choice([0, 1, 2, 3, 4], 500000),
    'is_payroll': np.random.choice([0, 1], 500000, p=[0.1, 0.9])
})

app_engagement_df = pd.DataFrame({
    'user_id': np.random.choice(range(1, 100001), 1000000),
    'session_date': pd.date_range(start='2022-03-01', periods=1000000, freq='5min'),
    'session_length_seconds': np.random.exponential(300, 1000000),
    'screens_viewed': np.random.poisson(5, 1000000)
})

transactions_df = pd.DataFrame({
    'transaction_id': range(1, 2000001),
    'user_id': np.random.choice(range(1, 100001), 2000000),
    'transaction_date': pd.date_range(start='2022-03-01', periods=2000000, freq='2min'),
    'transaction_amount': np.random.uniform(5, 200, 2000000),
    'merchant_category': np.random.choice(['grocery', 'restaurant', 'retail', 'utilities', 'other'], 2000000)
})

users_df = users_df[users_df['signup_date'] <= '2024-04-15']
direct_deposits_df = direct_deposits_df[(direct_deposits_df['deposit_date'] >= '2024-01-15') & (direct_deposits_df['deposit_date'] <= '2024-04-15')]
app_engagement_df = app_engagement_df[(app_engagement_df['session_date'] >= '2024-01-15') & (app_engagement_df['session_date'] <= '2024-04-15')]
transactions_df = transactions_df[(transactions_df['transaction_date'] >= '2024-01-15') & (transactions_df['transaction_date'] <= '2024-04-15')]

/var/folders/56/fsddj99x0bb1w5gqs1z1j95r0000gn/T/ipykernel_3558/1825391999.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'signup_date': pd.date_range(start='2022-01-01', periods=100000, freq='H'),


# Chime Feature Launch Case Study
Hello! I'm the Senior Data Analytics Manager at Chime. Today, we'd like to evaluate your technical and analytical skills by walking through a new feature launch scenario.

## Scenario: Early Direct Deposit Expansion
Chime is considering expanding our Early Direct Deposit (EDD) feature, which currently allows users to access their paychecks up to 2 days early. We're evaluating a potential enhancement that would offer select users access to their paychecks up to 4 days early instead of the standard 2 days.
We've run a controlled pilot with 50,000 users for the past 3 months, and we need to analyze the results to determine if we should roll this out more broadly.
Available Data
You have access to several tables:
- User information (demographics, account details)
- Direct deposit transactions
- App engagement data
- Card transaction history
- Customer service interactions

## Your Task

- First, help us evaluate if the 4-day early access feature is performing well. What metrics would you analyze?
- Use Python to perform an initial analysis of the feature's impact on:
    - User engagement
    - Spending behavior
    - Retention
- Identify any potential risks or concerns with expanding this feature.
- Make a recommendation: Should we roll this out to all users, continue testing, or abandon the initiative?

## Clarifying Question-Responses
- The data contains both our test group (users offered 4-day early access) and a control group (users with the standard 2-day early access). We have 3 months of data covering both groups.
- We selected 4 days based on competitive analysis and focus group feedback. Several competitors offer 3-day early access, and our user research suggested that pushing to 4 days would create a meaningful differentiation while still managing risk.
- Test and control groups were randomly assigned among eligible users (those with consistent direct deposit history of at least 6 months). We stratified the randomization to ensure balance across key demographics and usage patterns.
- Yes, the entire paycheck is accessible once we receive the deposit notification from the ACH network. There's no partial release - users get full access to their entire paycheck up to 4 days early in the test group.

# Framework

## Goal and Feature Information
Chime aims to provide financial freedom and build better financial habits for its users. In order to do this it uses features like EDD, which helps them get money earlier than their paydate. Traditionally it being available 2 days before paydate, we are changing it to 4 days for a cohort of users. 

## Clarifying Questions
- Does the data only contain trial data? I ask because why and how did we decide for the 4 days earlier config?
- Also, how are the test and control determined?
- Entire paycheck is accessible?

## Aligning on Goals of the exercise
Hypothesis on what would happen by moving form 2-day to 4-day

### Direct Impact
- Impact on feature adoption, increase in test group.
- Along same lines as above, proportion of users with payroll direct deposit would increase.
- Amount saved per user would increase, as more users would be using the feature.

### Indirect Impact
- We have no measure of interchange revenue, but a feature like this would increase that per user for the test group. As an indirect effect
- A proxy of measuring the interchange revenue increase, would be the app_engagement metrics. Increase in engagement, which is length of session, number of screens viewed, and also increase in retention, which is how many times the user come back in certain days, lets say a week
- We can also look at average number of transactions per month per user and average transaction amount per month per user. The test group show some lift.



In [82]:
user_direct_deposit_df = pd.merge(users_df, direct_deposits_df, how='inner', on='user_id')
user_direct_deposit_df.head(500)
print(len(user_direct_deposit_df))

2765


In [83]:
# Users who were present in the experiment but not registered for direct deposits
# Maybe we could target them with more marketing etc.
user_direct_deposit_df[pd.isnull(user_direct_deposit_df.deposit_id)]


Empty DataFrame
Columns: [user_id, signup_date, state, age_bucket, is_test_group, churn_date, deposit_id, deposit_date, deposit_amount, days_early, is_payroll]
Index: []

user_id  signup_date  state  age_bucket  is_test_group  churn_date  \
0       False        False  False       False          False        True   
1       False        False  False       False          False        True   
2       False        False  False       False          False        True   
3       False        False  False       False          False        True   
4       False        False  False       False          False        True   
...       ...          ...    ...         ...            ...         ...   
2760    False        False  False       False          False        True   
2761    False        False  False       False          False        True   
2762    False        False  False       False          False        True   
2763    False        False  False       False          False        True   
2764    False        False  False       False          False        True   

      deposit_id  deposit_date  deposit_amount  days_early  is_payroll  \
0          False         False           False       False       False   
1          False         False           False       False       False   
2          False         False           False       False       False   
3          False         False           False       False       False   
4          False         False           False       False       False   
...          ...           ...             ...         ...         ...   
2760       False         False           False       False       False   
2761       False         False           False       False       False   
2762       False         False           False       False       False   
2763       False         False           False       False       False   
2764       False         False           False       False       False   

      signup_month_date  deposit_month_date  
0                 False               False  
1                 False               False  
2                 False               False  
3                 False               False  
4                 False               False  
...                 ...                 ...  
2760              False               False  
2761              False               False  
2762              False               False  
2763              False               False  
2764              False               False  

[2765 rows x 13 columns]

In [98]:
def get_dd_metrics(group):
    ## proportion of users
    try:
        total_users = group['user_id'].nunique()
        if total_users==0:
            total_users = 1
        reqd_users = 0
        reqd_users_more_days = 0
        amount_deposited = 0
        reqd_users_df = pd.DataFrame()
        if group['is_test_group'].any()==0:
            reqd_users_df = group[(group['is_payroll']==1) & (group['days_early']<=2)]
            reqd_users = len(reqd_users_df)
        else:
            reqd_users_df = group[(group['is_payroll']==1) & (group['days_early']<=4)]
            reqd_users = reqd_users_df['user_id'].nunique()
            reqd_users_more_days_df = group[(group['is_payroll']==1) & (group['days_early']>2)]
            reqd_users_more_days = reqd_users_more_days_df['user_id'].nunique()
        amount_deposited = round(reqd_users_df['deposit_amount'].sum()/total_users,2)
        transactions_user = round(reqd_users_df['deposit_id'].nunique()/total_users,2)
        return pd.Series({
            'proportion_users': round(reqd_users*100/total_users,2)
            , 'proportion_users_more_days': round(reqd_users_more_days*100/total_users,2)
            , 'total_users': total_users
            , 'reqd_users': reqd_users
            , 'reqd_users_more_days': reqd_users_more_days
            , 'amount_deposited_per_user': amount_deposited
            , 'transactions_per_user': transactions_user
        })
        
    except Exception as e:
        print(f"Error in group {group.name}: {e}")
    
user_direct_deposit_df['signup_month_date'] = user_direct_deposit_df.signup_date.dt.strftime('%Y-%m-01')
user_direct_deposit_df['deposit_month_date'] = user_direct_deposit_df.deposit_date.dt.strftime('%Y-%m-01')
# grouped_metrics_df = user_direct_deposit_df.groupby(['is_test_group','signup_month_date','state','age_bucket','deposit_month_date']).apply(get_dd_metrics)
# grouped_metrics_df = user_direct_deposit_df.groupby(['is_test_group','deposit_month_date']).apply(get_dd_metrics)
# grouped_metrics_df = grouped_metrics_df.reset_index()
# grouped_metrics_df.columns = ['is_test_group','deposit_month_date','proportion_users','total_users','reqd_users','amount_deposited_per_user','transactions_per_user']
# pivot_df = grouped_metrics_df.pivot(index='is_test_group',columns='deposit_month_date',values=['proportion_users','amount_deposited_per_user','transactions_per_user'])
# print(grouped_metrics_df)
overall_proportion = user_direct_deposit_df.groupby(['is_test_group','deposit_month_date'], as_index=False).apply(get_dd_metrics)
print(overall_proportion)
# print(pivot_df)

   is_test_group deposit_month_date  proportion_users  \
0              0         2024-01-01             50.41   
1              0         2024-02-01             52.00   
2              0         2024-03-01             54.95   
3              0         2024-04-01             55.39   
4              1         2024-01-01             90.66   
5              1         2024-02-01             90.33   
6              1         2024-03-01             91.15   
7              1         2024-04-01             89.90   

   proportion_users_more_days  total_users  reqd_users  reqd_users_more_days  \
0                        0.00        244.0       123.0                   0.0   
1                        0.00        475.0       247.0                   0.0   
2                        0.00        444.0       244.0                   0.0   
3                        0.00        204.0       113.0                   0.0   
4                       35.80        257.0       233.0                  92.0   
5     

/var/folders/56/fsddj99x0bb1w5gqs1z1j95r0000gn/T/ipykernel_3558/1251022316.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  overall_proportion = user_direct_deposit_df.groupby(['is_test_group','deposit_month_date'], as_index=False).apply(get_dd_metrics)


In [94]:
user_direct_deposit_df['is_test_group'].value_counts(dropna=False)

is_test_group
0    1394
1    1371
Name: count, dtype: int64

In [46]:
# The small p-value in proportion differnce tells that test is significant, and also practicially significant

from statsmodels.stats.proportion import proportions_ztest
stat, pval = proportions_ztest([1158,748], [1203,1282], alternative='two-sided')
print(stat)
print(pval)


22.34228807500174
1.4350161388199674e-110


is_test_group  age_bucket
0              18-25         260
               26-35         283
               36-45         268
               46-55         273
               55+           285
1              18-25         266
               26-35         273
               36-45         245
               46-55         262
               55+           246
dtype: int64